# `nbconvert_html5` interfaces

this project exports tools for making jupyter notebook products more accessible. currently, this project features:
* an `nbconvert` template with that uses html5 semantics.

In [1]:
    from nbconvert import  get_exporter, get_export_names
    import nbconvert_html5, nbconvert, pathlib, bs4, traitlets
    from functools import partial
    from pathlib import Path
    EXPORTER = "a11y"

this module exports the `html5` nbconvert template.

In [2]:
    assert EXPORTER in get_export_names(), "the entry point is not defined."

the two templates - `html` and `html5` - can be loaded and compared.

In [3]:
    html, html5 = get_exporter("html")(), get_exporter(EXPORTER)()
    config = Path("jupyter_nbconvert_config.py")
    if "__file__" in locals(): config = Path(locals()["__file__"]).parent  / config        
    html5.update_config(traitlets.config.PyFileConfigLoader(str(config.absolute())).load_config())

## remediations

the major changes to `html5` version are that sections and articles have been added to the cells.

In [4]:
    A = html.from_filename("notebooks/lorenz-executed.ipynb")[0]
    B = html5.from_filename("notebooks/lorenz-executed.ipynb")[0]
    A, B = map(partial(bs4.BeautifulSoup, features="html.parser"), (A, B))

* adding a `main` landmark

In [5]:
    A.select("section"), len(B.select("main"))

([], 1)

* cells are articles

In [6]:
    A.select("article"), len(B.select("article.jp-Cell"))

([], 0)

* outputs are sections

## using the configuration to modify tests

`jupyter`s configuration system allows us to modify html representations on the fly.
see the `jupyter_nbconvert_config.py` for an example where we convert prompts to asides.

In [7]:
    # NBVAL_IGNORE_OUTPUT
    SOURCE = Path("notebooks/lorenz-executed.ipynb")
    TARGET = SOURCE.with_suffix(".html")
    !jupyter nbconvert --to $EXPORTER --config $config $SOURCE

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook notebooks/lorenz-executed.ipynb to a11y
[NbConvertApp] Writing 94575 bytes to notebooks/lorenz-executed.html


configuration files for manual testing knobs. each round of manual user testing we are testing isolated features. it would be hard, and inconsistent, to infer the documents the users are testing.

configuration files act as a way to record all the things we could be testing. it also allows for A/B testing where the configuration files store the inputs.

sometimes we are hindered by having too many fixes and too little knowledge of how well they will work. the flagging approach lets us tackle potential fixes independent of user testing. fixes are recorded with flags.